# Дипломная работа по курсу "Python для анализа данных"

Признаки:
1. satisfaction_level - Уровень удовлетворенности работой
2. Last_evaluation - Время с момента последней оценки в годах
3. number_projects - Количество проектов, выполненных за время работы
4. average_monthly_hours - Среднее количество часов на рабочем месте в месяц
5. time_spend_company - Стаж работы в компании в годах
6. work_accident - Происходили ли несчастные случаи на рабочем месте с сотрудником
7. left - уволился ли сотрудник
8. promotion_last_5years - повышался ли сотрудник за последние пять лет
9. department - отдел в котором работает сотрудник
10. salary - относительный уровень зарплаты

## 1 *Загрузите файл HR.csv в pandas dataframe*

In [ ]:
# импортируем модули
import numpy as np
import pandas as pd
import scipy.stats as st
import math
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as spy
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
# LDA-функционал
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# загржаем файл в DataFrame

df = pd.read_csv('HR.csv')

In [ ]:
df.head()

## 2. Рассчитайте основные статистики для переменных (среднее,медиана,мода,мин/макс,сред.отклонение).

In [ ]:
# gпосмотрим характеристики по всему датафрейму
df.describe()

разбиваем данные по типам

In [ ]:
# разбиваем на непрерывные, дискретные, категориальные

continues_titles = ['satisfaction_level', 'last_evaluation', 'average_montly_hours']
descrete_titles = ['number_project', 'time_spend_company']
nominal_titles = ['department', 'salary', 'Work_accident', 'left', 'promotion_last_5years']

получаем основные статистические характеристики

In [ ]:
def stat_values(continues_titles, descrete_titles, nominal_titles):
    for value in continues_titles:
        datetype='continues'
        print_value(value)
    for value in descrete_titles:
        datetype = 'descrete'
        print_value(value, datetype)
    for value in nominal_titles:
        datetype = 'nominal'
        print_value(value, datetype)

def print_value(value, datetype='continues'):
    print(f"\n.................\n{value}\n")
    if datetype == 'descrete':
        print(f"Среднее {df[value].mean():.0f}")
    else:        
        print(f"Мода {df[value].mode()[0]}")
    if datetype != 'nominal':
        print(f"Среднее {df[value].mean()}")     
        print(f"Максимум для {value} - {df[value].max()}")
        print(f"Минимум для {value} - {df[value].min()}")
        print(f"Размах {df[value].max() - df[value].min()}")          
        print(f"Медиана {df[value].median()}")   
        print(f"СКО {df[value].std()}")

In [ ]:
stat_values(continues_titles, descrete_titles, nominal_titles)

## 3. Рассчитайте и визуализировать корреляционную матрицу для количественных переменных. Определите две самые скоррелированные и две наименее скоррелированные переменные.

оставляем количественные непрерывные и дискретные данные

In [ ]:
data_for_cor = df[continues_titles + descrete_titles]
data_for_cor.head()

корреляционная матрица

In [ ]:
data_for_cor.corr()

коэффициент корреляции Спирмена

In [ ]:
data_for_cor.corr(method='spearman')

In [ ]:
# Возьмем матрицу коэффициентов корреляции

minmax = data_for_cor.corr()

# удаляем дубли

for title in abs(minmax).index:
    minmax[title][title:] = None

# Вычислим 

new_array = np.array([])
for name, values in minmax.iteritems():
    new_array = np.append(new_array, values)

# убираем NaN. Сортируем массив.

new_array = new_array[np.isfinite(new_array)]
new_array = np.sort(new_array)

# берем 2-е элементы с конца и начала

second_max = new_array[-2]
second_min = new_array[1]

# фильтруем данные 

two_max = minmax[minmax.loc[:] >= second_max]
two_min = minmax[minmax.loc[:] <= second_min]

In [ ]:
# Максимальная корреляция:

print('Максимальная корреляция:')

two_max.dropna(axis=0, how='all').dropna(axis=1, how='all')

In [ ]:
# Минимальная корреляция:

print('Минимальная корреляция:')

two_min.dropna(axis=0, how='all').dropna(axis=1, how='all')

строим haetmap

In [ ]:
sns.heatmap(data_for_cor.corr(), annot=True)

по таблице бОльшая корреляция для **number_project** и **average_montly_hours**. Вероятно, чем больше сотрдуник реализовал проектов, тем больше вермени он проводил на работе.
Также есть бОльшая корреляция между **number_project** и **last_evaluation**.

Самая низкая корреляция между **satisfaction_level** и **average_montly_hours**, а также между **satisfaction_level** и **time_spend_company**

## 4. Рассчитайте сколько сотрудников работает в каждом департаменте.

для начала исключим всех уволившихся сотрудников

In [ ]:
df['left'].value_counts().head()

предполгается, что 0 - уволившиеся сотрудники, 1 - работающие. Если компания существует долго, то количество уволившихся сотрудников значительно выше.

In [ ]:
employee = df[df['left'] == 1]

In [ ]:
employee.info()

Получаем dataframe с количеством  сотрудников, работающих в каждом департаменте

In [ ]:
employee_by_dept = employee[['left', 'department']].groupby(by='department').count()
employee_by_dept.columns = ['emp_count']

In [ ]:
employee_by_dept

In [ ]:
employee_by_dept.plot(kind='bar')

## 5. Показать распределение сотрудников по зарплатам.

In [ ]:
employee_by_salary = employee[['left', 'salary']].groupby(by='salary').count().sort_index()
employee_by_salary.columns = ['emp_count']
employee_by_salary

In [ ]:
employee_by_salary.plot(kind='bar', cmap='autumn')

## 6. Показать распределение сотрудников по зарплатам в каждом департаменте по отдельности

Построим сводную таблицу

In [ ]:
employee.pivot_table(index = 'department', columns = 'salary', values = 'left', aggfunc = 'count', fill_value = 0, 
                    margins = True).style.bar(align='mid')

## 7. Проверить гипотезу, что сотрудники с высоким окладом проводят на работе больше времени, чем сотрудники с низким окладом

In [ ]:
print('''Получим DataFrame с нужными данными: 
1) оставляем столбцы "average_montly_hours" и "salary"
2) "salary" по заданию сравнивается только высокий оклад и низкий. "medium" исключаем''')
sal_vs_amh = df[['average_montly_hours', 'salary']][df['salary'].isin(['low', 'high'])]
sal_vs_amh.head()

In [ ]:
print('Количество данных датафрейма')
sal_vs_amh.groupby(by='salary').count().T

In [ ]:
print('Среднее')
sal_vs_amh.groupby(by='salary').mean()

In [ ]:
print('Посмотрим графики')
low = sal_vs_amh[sal_vs_amh['salary'] == 'low']['average_montly_hours']
print('Дисперсия', low.var())
high = sal_vs_amh[sal_vs_amh['salary'] == 'high']['average_montly_hours']
medium = sal_vs_amh[sal_vs_amh['salary'] == 'medium']['average_montly_hours']
print('Дисперсия', high.var())
plt.hist(low, bins=20)
plt.hist(high, bins=20)

In [ ]:
sns.violinplot(x=sal_vs_amh['average_montly_hours'], y=sal_vs_amh['salary'])

In [ ]:
sns.boxplot(x=sal_vs_amh['average_montly_hours'], y=sal_vs_amh['salary'])

In [ ]:
print(f'Дисперсия для маленькой зарплаты - {low.var()}, дисперсия большой зарпалты - {high.var()}')

Дисперсии не сильно отличаются

In [ ]:
import scipy as spy
from scipy import stats
from scipy.stats import ttest_rel

расчитываем среднее по уровням зарплат

In [ ]:
for salary_value in sal_vs_amh['salary'].unique():
    print(f'Зарплата уровня - {salary_value}, среднее количество часов на рабочем месте в месяц - {sal_vs_amh[sal_vs_amh["salary"] == salary_value]["average_montly_hours"].mean():.3f}')

Выполним тест Стьюдента

Допущения

Наблюдения внутри каждой выборки независимы друг от друга.
Распределение данных - нормлаьное или близкое к нормальному. Можно применять для других распределений, но осторожно.
Одинаковая вариация в выборках.
Гипотеза

H0: Среднее время на работе для разных зарплат совпадают

H1: Среднее время на работе для разных зарплат отличаются

In [ ]:
data1 = sal_vs_amh[sal_vs_amh['salary'] == 'low']['average_montly_hours']
data2 = sal_vs_amh[sal_vs_amh['salary'] == 'high']['average_montly_hours']
alpha = 0.05

stat, p = st.ttest_ind(data1, data2)
stat1, p1 = st.mannwhitneyu(data1, data2)
print(f"Статистика t-теста = {stat:.5f}, p = {p:.5f}")

if p > alpha:
    print('H0: Не отклоняем нулевую гипотезу, средние, вероятно, для разных зарплат совпадают')
else:
    print('H1: Отклоняем нулевую гипотезу, средние, вероятно, различаются')


## 8. Рассчитать следующие показатели среди уволившихся и не уволившихся сотрудников (по отдельности):


*   Доля сотрудников с повышением за последние 5 лет
*   Средняя степень удовлетворенности
*   Среднее количество проектов



*Доля сотрудников с повышением за последние 5 лет*

In [ ]:
# посмотрим какие уникальные значения имеются для promotion_last_5years

df['promotion_last_5years'].unique()

In [ ]:
# создадим датафрейм нужных столбцов, используя группировку

left = df[['left', 'promotion_last_5years', 'salary']].groupby(['left', 'promotion_last_5years'], as_index=False).count()
left.columns = ['left', 'promotion_last_5years', 'count']
left

In [ ]:
def share(left):
    '''
    Вычисляет долю от общего количества сотрудников
    '''
    total_sum = left['count'].sum()
    left['share'] = left['count'] / total_sum
    return left

In [ ]:
# Вычисляем долю(share) сотрудников с повышением за последние 5 лет по уволившимся и работающим сотрдуникам

left.groupby('left').apply(share)

*Средняя степень удовлетворенности* 

In [ ]:
# создадим датафрейм нужных столбцов для satisfaction_level, используя группировку

satisfaction_level = df[['left', 'satisfaction_level']].groupby(['left'], as_index=False).mean()
satisfaction_level.columns = ['left', 'average_satisfaction_level']
print(satisfaction_level)  

*Среднее количество проектов*

In [ ]:
# создадим датафрейм нужных столбцов для number_project, используя группировку

number_project = df[['left', 'number_project']].groupby(['left'], as_index=False).mean()

number_project.columns = ['left', 'average_number_project']

# посчитаем, что нужно целое количество проектов. Округлим и преобразуем в int

number_project['average_number_project'] = number_project['average_number_project'].round(2)

print(number_project) 

## 9. Разделить данные на тестовую и обучающую выборки. Построить модель LDA, предсказывающую уволился ли сотрудник на основе имеющихся факторов (кроме department и salary). Оценить качество модели на тестовой выборки.

In [ ]:
# посмторим еще раз на данные.

df

Проанализируем признаки:

1. satisfaction_level - Уровень удовлетворенности работой - *непрерывная величина*
2. Last_evaluation  - Время с момента последней оценки в годах - *непрерывная величина*
3. number_projects - Количество проектов, выполненных за время работы - *дикретная количественная величина*
4. average_monthly_hours - Среднее количество часов на рабочем месте в месяц - *дикретная количественная величина *
5. time_spend_company - Стаж работы в компании в годах - *дикретная количественная величина*
6. Work_accident - Происходили ли несчастные случаи на рабочем месте с сотрудником - *категориальная величина*
7. left - уволился ли сотрудник - *категориальная величина*
8. promotion_last_5years - повышался ли сотрудник за последние пять лет - *категориальная величина*
9. department  - отдел в котором работает сотрудник - *категориальная величина*
10. salary - относительный уровень зарплаты - *категориальная величина*

In [ ]:
# исключаем столбцы, оставив только количественные для корректной работы модели
data = df[['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company']]

In [ ]:
data.info()

In [ ]:
# данные по классам(left)

data_class = df['left']
data_class.value_counts()

In [ ]:
# разделяем выборку на тестовую и обучающую

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, data_class, test_size=0.25, random_state=42)

In [ ]:
print(f'Тренировочные данные - {len(y_train)}\nданные для тестирования - {len(y_test)}')

In [ ]:
# обучаем данные

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

In [ ]:
# делаем прогноз на тестовой выборке
pd.DataFrame(lda.predict(X_test))

In [ ]:
# смотрим разницу факта и прогноза

y_test.reset_index(drop=True, inplace= True)
result = pd.DataFrame([y_test, lda.predict(X_test)]).T
result.columns = ['left_test', 'left_predict']
result.info()

In [ ]:
# расчет точности accuracy
from sklearn.metrics import accuracy_score

accuracy_score(y_test, lda.predict(X_test))

In [ ]:
# коэффициенты дискриминатных линий

lda.coef_

In [ ]:
# применим метод кросс-валидации. 
# 10 сгибов 3 пвоторения

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

# оценка модели

scores = cross_val_score(lda, X_train, y_train, scoring='accuracy', cv=cv)
print(np.mean(scores)) 

In [ ]:
# сравним с логистической регрессией

data1 = df[['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']]

X_train1, X_test1, y_train, y_test = train_test_split(data1, data_class, test_size=0.25)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train1, y_train)
lr.predict(X_test1)

In [ ]:
# вероятность принадлежности к тому или иному классу

lr.predict_proba(X_test1)

In [ ]:
accuracy_score(y_test, lr.predict(X_test1))

In [ ]:
lr.coef_